In [1]:
%load_ext autoreload
%autoreload 2

In [52]:
import numpy as np
import pandas as pd

from src.HyperHeuristic import HyperHeuristic
from src.stop.NoImprovement import NoImprovement
from src.stop.MaxRuntime import MaxRuntime
from src.weights.SegmentedWeights import SegmentedWeights
from src.accept.SimulatedAnnealing import SimulatedAnnealing
from src.accept.HillClimbing import HillClimbing
from src.initial_solution.RandomConstructive import RandomConstructive
from src.operators.DestroyOperators import RandomRemove, WorstRemove, ShawRemove 
from src.operators.RepairOperators import RandomRepair, GreedyRepair, RegretInsertion
from src.local_search.FirstImprovement import FirstImprovement
from src.local_search.BestImprovement import BestImprovement
from src.KnapsackInstance import KnapsackInstance
from utils.benchmark import BinaryKnapsackBenchmark

In [53]:
benchmarkPath = "../backup/data/benchmark/binaryKnapsack"
benchmark = BinaryKnapsackBenchmark("binary", benchmarkPath)
instancesList = benchmark.getInstancesList()

In [110]:
columns = ['instanceName', 'optimumFO', 'FO', 'relativeError', 'runtime']
values = []

for i in range(10):
    # Create Hyper-Heuristic
    hyperHeuristic = HyperHeuristic()

    randomRemove1 = RandomRemove(0.2)
    randomRemove2 = RandomRemove(0.4)
    worstRemove1 = WorstRemove(0.2)
    worstRemove2 = WorstRemove(0.4)
    randomRepair = RandomRepair()
    greedyRepair = GreedyRepair()
    firstImprovement = FirstImprovement()

    hyperHeuristic.add_refinement(randomRemove1, 'randomRemove0.2')
    hyperHeuristic.add_refinement(randomRemove2, 'randomRemove0.4')
    hyperHeuristic.add_refinement(worstRemove1, 'worstRemove0.2')
    hyperHeuristic.add_refinement(worstRemove2, 'worstRemove0.4')
    hyperHeuristic.add_refinement(randomRepair, 'randomRepair')
    hyperHeuristic.add_refinement(greedyRepair, 'greedyRepair')
    hyperHeuristic.add_refinement(firstImprovement, 'firstImprovement')

    randomConstruct = RandomConstructive()

    simulatedAnnealingCriteria = SimulatedAnnealing(
        start_temperature = 100,
        end_temperature = 0.001,
        step = 0.99,
        method = 'exponential'
    )

    # hillClimbing = HillClimbing()

    hyperHeuristic.add_acceptance(simulatedAnnealingCriteria, 'simulatedAnnealingCriteria')
    # hyperHeuristic.add_acceptance(hillClimbing, 'simulatedAnnealingCriteria')

    segWeights = SegmentedWeights(
        scores = [5,4,1,0.5],
        refinements = hyperHeuristic.refinement,
        acceptances = hyperHeuristic.acceptance,
        seg_decay = 0.8,
        seg_length = 10
    )

    # stop = NoImprovement(max_iterations = 10)
    stop = MaxRuntime(45)

    # Run instance
    instanceName = np.random.choice(instancesList)
    instance = benchmark.parseInstance(instanceName)
    optimum = benchmark.getOptimalFO(instanceName)

    result = hyperHeuristic.iterate(instance, randomConstruct, segWeights, stop)
    objective = result.best_state.objective(isMinimizing = False)
    runtime = result.statistics.total_runtime
    
    values.append([
        instanceName.replace('/test.in',''),
        optimum,
        objective, 
        (optimum - objective)/optimum,
        runtime
    ])

summary = pd.DataFrame(data = values, columns = columns)
summary.head()

/mnt/c/Users/caina/OneDrive/Área de Trabalho/UFRJ/Mestrado/OTIMIA/hyper-heuristics/src/accept/SimulatedAnnealing.py:85: RuntimeWarning: overflow encountered in exp
  probability = np.exp(
/mnt/c/Users/caina/OneDrive/Área de Trabalho/UFRJ/Mestrado/OTIMIA/hyper-heuristics/src/accept/SimulatedAnnealing.py:85: RuntimeWarning: overflow encountered in exp
  probability = np.exp(
/mnt/c/Users/caina/OneDrive/Área de Trabalho/UFRJ/Mestrado/OTIMIA/hyper-heuristics/utils/instancesRepresentation.py:76: RuntimeWarning: overflow encountered in long_scalars
  foValue -= noFeasiblePenalty * (totalKnapsacksWeights[knapsack] - knapsacksCapacities[knapsack])
/mnt/c/Users/caina/OneDrive/Área de Trabalho/UFRJ/Mestrado/OTIMIA/hyper-heuristics/src/accept/SimulatedAnnealing.py:85: RuntimeWarning: overflow encountered in exp
  probability = np.exp(
/mnt/c/Users/caina/OneDrive/Área de Trabalho/UFRJ/Mestrado/OTIMIA/hyper-heuristics/src/accept/SimulatedAnnealing.py:85: RuntimeWarning: overflow encountered in exp


,instanceName,optimumFO,FO,relativeError,runtime
0,n_1200_c_1000000_g_14_f_0.3_eps_0.1_s_100,1006688,966066,4.035212e-02,45.024690
1,n_400_c_10000000000_g_2_f_0.1_eps_1e-05_s_100,5000101912,5000101854,1.159976e-08,45.017430
2,n_600_c_1000000_g_14_f_0.2_eps_0.01_s_100,1005968,994241,1.165743e-02,45.003099
3,n_800_c_10000000000_g_14_f_0.1_eps_0.01_s_200,-1,9953133944,9.953134e+09,45.058896
4,n_400_c_100000000_g_2_f_0.2_eps_0.0001_s_100,50014064,50014016,9.597300e-07,45.000872


In [112]:
summary.to_csv('summary.csv', index = True)